In [1]:
import csv
import pandas as pd
import numpy as np

In [68]:
df = [pd.DataFrame() for _ in range(5)]

for x in range(1, 6):
    with open(f'percentuale0{x}.csv', 'r') as file:
        dft = pd.read_csv(file)
        dft[f'comune{x}'] = dft['Comune;Percentuale'].str.split(';').str[0]
        dft[f'percentuale{x}'] = dft['Comune;Percentuale'].str.split(';').str[1].astype(float)
        dft.drop(columns=['Comune;Percentuale'], inplace=True)
        df[x-1] = dft

df = pd.concat(df, axis=1)
df.drop(columns=['comune2', 'comune3', 'comune4', 'comune5'], inplace=True)
df.rename(columns={'comune1': 'comune'}, inplace=True)
df

,comune,percentuale1,percentuale2,percentuale3,percentuale4,percentuale5
0,ACQUI TERME (ALESSANDRIA),31.47,31.44,31.46,31.47,31.42
1,ALBERA LIGURE (ALESSANDRIA),32.28,32.28,32.28,32.28,32.28
2,ALESSANDRIA (ALESSANDRIA),30.47,30.45,30.47,30.47,30.43
3,ALFIANO NATTA (ALESSANDRIA),30.07,30.07,30.07,30.07,30.07
4,ALICE BEL COLLE (ALESSANDRIA),25.57,25.57,25.57,25.57,25.57
...,...,...,...,...,...,...
7891,URI (SASSARI),32.28,32.28,32.28,32.28,32.24
7892,USINI (SASSARI),25.71,25.71,25.71,25.71,25.71
7893,VALLEDORIA (SASSARI),16.12,16.10,16.12,16.12,16.12
7894,VIDDALBA (SASSARI),19.79,19.79,19.79,19.79,19.79


In [69]:
for x in range(1, 5):
    df[f'gap_{x}_to_5'] = df[f'percentuale{x}'] - df['percentuale5']

df.drop(columns=['percentuale1', 'percentuale2', 'percentuale3', 'percentuale4', 'percentuale5'], inplace=True)
df['provincia'] = df['comune'].str.split('(').str[1].str.split(')').str[0]

df

,comune,gap_1_to_5,gap_2_to_5,gap_3_to_5,gap_4_to_5,provincia
0,ACQUI TERME (ALESSANDRIA),0.05,0.02,0.04,0.05,ALESSANDRIA
1,ALBERA LIGURE (ALESSANDRIA),0.00,0.00,0.00,0.00,ALESSANDRIA
2,ALESSANDRIA (ALESSANDRIA),0.04,0.02,0.04,0.04,ALESSANDRIA
3,ALFIANO NATTA (ALESSANDRIA),0.00,0.00,0.00,0.00,ALESSANDRIA
4,ALICE BEL COLLE (ALESSANDRIA),0.00,0.00,0.00,0.00,ALESSANDRIA
...,...,...,...,...,...,...
7891,URI (SASSARI),0.04,0.04,0.04,0.04,SASSARI
7892,USINI (SASSARI),0.00,0.00,0.00,0.00,SASSARI
7893,VALLEDORIA (SASSARI),0.00,-0.02,0.00,0.00,SASSARI
7894,VIDDALBA (SASSARI),0.00,0.00,0.00,0.00,SASSARI


In [70]:
# cumulate the gaps for each province
df['gap_1_to_5'] = df.groupby('provincia')['gap_1_to_5'].transform('sum')
df['gap_2_to_5'] = df.groupby('provincia')['gap_2_to_5'].transform('sum')
df['gap_3_to_5'] = df.groupby('provincia')['gap_3_to_5'].transform('sum')
df['gap_4_to_5'] = df.groupby('provincia')['gap_4_to_5'].transform('sum')
# drop duplicates to keep only one row per province
df.drop_duplicates(subset='provincia', keep='first', inplace=True)
df.drop(columns=['comune'], inplace=True)
df.reset_index(drop=True, inplace=True)
df['medium_gap'] = df[['gap_1_to_5', 'gap_2_to_5', 'gap_3_to_5', 'gap_4_to_5']].mean(axis=1)
df

,gap_1_to_5,gap_2_to_5,gap_3_to_5,gap_4_to_5,provincia,medium_gap
0,2.98,3.85,7.72,7.73,ALESSANDRIA,5.5700
1,-1.51,-4.46,-3.13,-2.31,ASTI,-2.8525
2,-3.42,-3.00,-2.77,-2.84,BIELLA,-3.0075
3,-3.06,-4.75,-3.37,-2.22,CUNEO,-3.3500
4,-1.54,-1.64,-0.06,0.15,NOVARA,-0.7725
...,...,...,...,...,...,...
101,0.25,0.15,0.21,0.25,TRAPANI,0.2150
102,1.92,1.73,1.81,1.59,CAGLIARI,1.7625
103,1.02,0.99,1.04,1.89,NUORO,1.2350
104,1.69,1.59,1.82,1.60,ORISTANO,1.6750


In [72]:
df.to_csv('risultato_finale.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)